MAIN CODE

In [39]:
import requests
import datetime

thunder1=[] #name
thunder2=[] #price
location = []
description = []
time = []


for page_number in range(0,120): 

    url = "https://api.bunjang.co.kr/api/1/find_v2.json?q=%EC%95%84%EC%9D%B4%ED%8F%B011&order=score&page={0}&request_id=2022519113331&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4".format(page_number)
    res = requests.get(url)
    res.encoding=None
    data = res.json()
    
    for d in data['list']:
        name = d['name']
        loc = d['location']
        price = int(d['price'])
        pid = d['pid']
        unix = str(datetime.datetime.fromtimestamp(d['update_time']))
        unix = unix[0:7]

        
        excepts = ["케이스","삽니다", "매입", "구매", "구입","프로","Pro","pro", "미니", "PRO", "11PRO"]
        targets = ['배터리', '판매', '팔아요']

        if (price >= 50000) & (price < 1600000) ==True :
            if ( d['ad']==False ):
                if any(x in d["name"] for x in targets) and not any(x in d['name'] for x in excepts):
                    thunder1.append(d['name'])
                    thunder2.append(price)
                    time.append(unix)
                    loc = loc.split(' ')
                    loc = loc[0]
                    # location.append(loc)
                    if len(loc) == 0:
                        location.append('NULL')
                    else:
                        location.append(loc)

                    url2 = 'https://api.bunjang.co.kr/api/1/product/{0}/detail_info.json?version=4'.format(pid)
                    res = requests.get(url2)
                    res.encoding=None
                    data = res.json()
                
                    description.append(data['item_info']['description'])
                    


                    print(name, price, unix, loc)

        else:
            pass

# print(thunder1,thunder2, time, location )


40578 아이폰11 퍼플 256GB 배터리93% 44만원 울산중고폰 440000 2022-05 울산광역시
60026 아이폰11 화이트 128GB 배터리94% 40만원 울산중고폰 400000 2022-05 울산광역시
59778 아이폰11 화이트 64GB 배터리94% 36만원 판매울산중고폰 360000 2022-05 울산광역시
59139 아이폰11 블랙 64GB 배터리94% 36만원 판매 울산중고폰 360000 2022-05 울산광역시
아이폰11 화이트 64기가 A급 배터리성능97 팔아요  380000 2022-05 경기도
40578 아이폰11 퍼플 256GB 배터리93% 44만원 울산중고폰 440000 2022-05 울산광역시
60026 아이폰11 화이트 128GB 배터리94% 40만원 울산중고폰 400000 2022-05 울산광역시
아이폰11 64기가 초S급 판매합니다 340000 2022-05 충청북도
아이폰11 퍼플 128기가 액정무기스 무잔상 [배터리83% 03935] 400000 2022-05 부산광역시
(405) 아이폰11 128G 블랙 최저가판매 390000 2022-05 경기도
아이폰11 128기가 판매합니다 390000 2022-05 경기도
아이폰11 128G + 에어팟1세대 판매 370000 2022-05 
애플 아이폰11 128GB 배터리94퍼 팝니다 울산중고폰  430000 2022-05 울산광역시
퍼플 아이폰11 판매 128g 박스포함 460000 2022-05 인천광역시
아이폰11 블랙 128GB 판매 360000 2022-05 서울특별시
아이폰 11 128 블랙 배터리92% ) 85116 420000 2022-05 인천광역시
아이폰11 액정 부품 (정품추출) 팔아요. 70000 2022-05 경기도
아이폰11 64 배터리100% 옐로우 ) 63666 390000 2022-05 인천광역시
아이폰11 64 화이트 배터리100% 팝니다! )43597 360000 2022-05 인천광역시
아이폰11 64기가 판매해요  350000 2022-

Desc 에서 배터리 효율 뽑아내기

In [40]:
import re
from bs4 import BeautifulSoup

li = []



for i in range(len(description)):
    text = description[i]
    regex = '배터리\s*.*\d{2,3}'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    text = str(res)
    res2 = pattern2.findall(text)
    if len(res2) == 0:
        li.append('NULL')
    else:
        li.append(res2[0])

print(li)



'''
for i in range(len(li)):    
    text = li[i]
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    res2 = pattern2.findall(text)
    li1.append(res2)
'''
#print(li1)

['93', '94', '94', '94', '97', '93', '94', '80', '83', 'NULL', '87', 'NULL', '94', 'NULL', '78', '92', 'NULL', '100', '100', '84', '100', 'NULL', '86', 'NULL', '99', '87', 'NULL', 'NULL', '92', '81', 'NULL', 'NULL', 'NULL', '81', '79', 'NULL', '83', 'NULL', '100', '86', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '96', 'NULL', 'NULL', '100', '81', 'NULL', '100', '83', 'NULL', 'NULL', '85', 'NULL', '94', '94', '83', 'NULL', 'NULL', '78', 'NULL', '80', 'NULL', '84', '85', 'NULL', 'NULL', '87', '091', '91', '82', '82', '82', '100', '81', 'NULL', '86', 'NULL', '010', 'NULL', 'NULL', '86', '90', '85', '83', 'NULL', '100', '79', '87', '83', '93', '83', '75', 'NULL', 'NULL', '100', '087', 'NULL', 'NULL', '010', '85', '78', '89', '93', '84', '089', '79', '89', 'NULL', '98', 'NULL', 'NULL', 'NULL', 'NULL', '87', '085', '84', '80', '85', '77', '093', 'NULL', 'NULL', 'NULL', '84', '091', 'NULL', '93', '84', 'NULL', '82', '84', '86', 'NULL', '89', '88', '90', 'NULL', '097', '86', '90', 'NULL',

"\nfor i in range(len(li)):    \n    text = li[i]\n    regex2 = '\\d{2,3}'\n    pattern2 = re.compile(regex2)\n    res2 = pattern2.findall(text)\n    li1.append(res2)\n"

용량 빼내기

In [41]:
size = []

for i in range(len(description)):
    text = description[i]
    regex = '64|128'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    if len(res) == 0:
        size.append('NULL')
    else:
        size.append(res[0])
print(size)

['NULL', '128', '64', '64', '64', 'NULL', '128', '64', '128', '128', 'NULL', 'NULL', '128', 'NULL', 'NULL', '128', 'NULL', '64', '64', 'NULL', '64', 'NULL', 'NULL', 'NULL', '128', 'NULL', 'NULL', 'NULL', '64', 'NULL', 'NULL', '64', 'NULL', 'NULL', 'NULL', '128', '64', '64', '128', '64', '128', '128', '128', '128', 'NULL', '128', 'NULL', 'NULL', '64', 'NULL', '64', 'NULL', '64', '64', 'NULL', '128', '64', '64', '128', '128', '64', '64', '64', '128', 'NULL', '64', 'NULL', 'NULL', '128', '128', '128', '128', '128', '128', '64', '64', '64', '128', '64', 'NULL', 'NULL', '64', '64', '64', 'NULL', '64', '64', '128', '128', 'NULL', '128', 'NULL', '128', '128', '64', '128', 'NULL', 'NULL', 'NULL', '128', '128', 'NULL', 'NULL', '64', '128', 'NULL', 'NULL', '64', '128', '128', '128', 'NULL', '64', 'NULL', '64', '128', '64', '64', '128', '128', '64', 'NULL', '128', '64', '128', '128', 'NULL', 'NULL', '128', '128', 'NULL', 'NULL', '128', 'NULL', '64', '128', 'NULL', '128', '128', '64', '64', '64', 

판다스로 데이터프레임 만들기

In [42]:
import pandas as pd
df = pd.DataFrame({'name' : thunder1,
                 'price' : thunder2,
                 'time' : time,
                  'loction' : location,
                  'battery' : li,
                  'size': size
                  })
df

,name,price,time,loction,battery,size
0,40578 아이폰11 퍼플 256GB 배터리93% 44만원 울산중고폰,440000,2022-05,울산광역시,93,NULL
1,60026 아이폰11 화이트 128GB 배터리94% 40만원 울산중고폰,400000,2022-05,울산광역시,94,128
2,59778 아이폰11 화이트 64GB 배터리94% 36만원 판매울산중고폰,360000,2022-05,울산광역시,94,64
3,59139 아이폰11 블랙 64GB 배터리94% 36만원 판매 울산중고폰,360000,2022-05,울산광역시,94,64
4,아이폰11 화이트 64기가 A급 배터리성능97 팔아요,380000,2022-05,경기도,97,64
...,...,...,...,...,...,...
650,아이폰 IP 11 128GB A급 판매합니다~,410000,2022-05,서울특별시,NULL,128
651,아이폰 IP 11 128GB A급 판매합니다~,410000,2022-05,서울특별시,NULL,128
652,아이폰 IP 11 128GB A급 판매합니다~,410000,2022-05,서울특별시,NULL,128
653,아이폰 IP 11 128GB A급 판매합니다~,410000,2022-05,서울특별시,NULL,128


데이터 엑셀파일로 추출

In [43]:
from pandas import ExcelWriter
df.to_excel('C:/Users/admin/Desktop/big/thunder.xlsx', index=False)